In [6]:
import pandas as pd

In [7]:
historical_stock_prices = pd.read_csv('historical_stock_prices.csv')

In [8]:
historical_stocks = pd.read_csv('historical_stocks.csv')

In [9]:
historical_stock_prices.head()

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14


In [10]:
historical_stocks.head()

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS


In [11]:
combined_stocks_df = pd.merge(historical_stock_prices, historical_stocks, on='ticker', how='left')

In [12]:
sorted_date = combined_stocks_df.sort_values(by='date', ascending=False)

In [ ]:
sorted_date.head(10)

In [14]:
grouped_stocks_ticker = combined_stocks_df.groupby('ticker')

In [15]:
max_date_df = combined_stocks_df.loc[combined_stocks_df['date'] == '2018-08-24']

In [16]:
currently_trading_ticker = max_date_df = combined_stocks_df.loc[combined_stocks_df['date'] == '2018-08-24']['ticker']

In [17]:
currently_trading_ticker = currently_trading_ticker.values

In [18]:
filtered_stocks = pd.DataFrame(columns=('ticker', 'open', 'close', 'adj_close', 'low', 'high', 'volume', 'date', 'exchange', 'name', 'sector', 'industry'))

In [19]:
filtered_stocks

,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry


In [ ]:
for index, row in combined_stocks_df.iterrows():
    if row[0] in currently_trading_ticker:
        filtered_stocks.append(combined_stocks_df.iloc[index,:])

In [ ]:
currently_trading_df = combined_stocks_df[combined_stocks_df['ticker'].isin(currently_trading_ticker)]

In [ ]:
currently_trading_df.head()